### Se realizaron dos implementaciones del método de reservoir sampling, la primer aproximación aunque sencilla de implementar y leer, su complejidad es $O(n)$, donde $n$ es el tamaño total (y desconocido de la población) por lo que fue muy tardado su ejecución y se procedió a realizar la segunda implementación.

# Implementación naive 

In [1]:
import pandas as pd
import random as ran

In [2]:
ran.seed(0) # fijamos la semilla del generador de numeros aleatorio 
m = 50*1000 # numero de registros dentro de nuestra muestra uniforme
#file_to_read = 'checkouts-by-title.csv' #path del archivo a leer
file_to_read = 'checkouts-by-title.csv' #path del archivo a leer

In [3]:
sample_init = pd.read_csv(file_to_read, nrows = m)

In [4]:
sample_init.shape

(50000, 11)

In [5]:
index = sample_init.shape[0]
k = sample_init.columns
while  1>0:
    # lectura hasta fin de archivo 
    try:
        # lectura lineal de un solo registro
        temp = pd.read_csv(file_to_read, nrows= 1, skiprows = lambda x: x in range(0, index) , header=0)
        # comienza sub muestreo aleatorio  
        j = ran.randint(0, index+1)
        if(index %1000 ==0 ):
                print('index  :' + str(index))
        if j <=m :
            for columna in range(len(k)):
                sample_init.loc[j, k[columna]]  = temp.iloc[0, columna] 
        # termina submuestreo aleatorio
        index+=1
        continue
    except:
        print(index)
        print('EOF :D')
        break

index  :50000
index  :51000
index  :52000
52029
EOF :D


In [6]:
sample_init.head()

,UsageClass,CheckoutType,MaterialType,CheckoutYear,CheckoutMonth,Checkouts,Title,Creator,Subjects,Publisher,PublicationYear
0,Physical,Horizon,BOOK,2006,6,1,McGraw-Hill's dictionary of American slang and...,"Spears, Richard A.",English language United States Slang Dictionar...,"McGraw-Hill,",c2006.
1,Physical,Horizon,BOOK,2006,6,1,"Emma, Lady Hamilton / Flora Fraser.","Fraser, Flora","Hamilton Emma Lady 1761 1815, Nelson Horatio N...","Knopf : Distributed by Random House,","1987, c1986."
2,Physical,Horizon,BOOK,2006,6,2,Red midnight,NaN,"Survival Fiction, Emigration and immigration F...",NaN,NaN
3,Physical,Horizon,BOOK,2006,6,1,Just the financial facts how to identify nugge...,NaN,Investments Information services,NaN,NaN
4,Physical,Horizon,BOOK,2006,7,1,Seven sisters,NaN,"Mystery fiction, California Fiction, Harper Be...",NaN,NaN


In [7]:
sample_init.to_csv('sample_size_naive'+ str(m) + file_to_read, index = False)

# Implementación algoritmo $L$ 
La segunda implementación que se realizo fue la del algoritmo L, cuyo complejidad es $O(k * log(n/k))$ donde $k$ es el tamaño de la muestra que buscamos. Esta implementación puede no ser optima, pero el algoritmo permitió generar la muestra con la que se construyo el reporte en un tiempo razonable. 

In [8]:
import pandas as pd
import random as ran
import numpy as np
ran.seed(0) # fijamos la semilla del generador de numeros aleatorio 
m = 2*10**6 # numero de registros dentro de nuestra muestra uniforme
#file_to_read = 'checkouts-by-title.csv' #path del archivo a leer
file_to_read = 'checkouts-by-title.csv' #path del archivo a leer
sample_init = pd.read_csv(file_to_read, nrows = m)

In [9]:
index = sample_init.shape[0]
k = sample_init.columns
i = m

w = np.exp ( np.log( ran.uniform(0, 1)) / m )
print('w1:    ' + str(w))
while  1>0:
    # lectura hasta fin de archivo 
    i = i + int( np.log( ran.uniform(0, 1) ) / np.log(1- w) ) + 1 
    try:
        # lectura lineal de un solo registro
        temp = pd.read_csv(file_to_read, nrows= 1, skiprows = lambda x: x in range(0, i) , header=0)
        # comienza sub muestreo aleatorio  
        j = ran.randint(0, m)
        if i %1000 == 0:
            print('index  :' + str(i))
        for columna in range(len(k)):
            sample_init.loc[j, k[columna]]  = temp.iloc[0, columna] 
        # termina submuestreo aleatorio
        w = w * np.exp ( np.log( ran.uniform(0, 1)) / m )
        continue
    except:
        print('EOF :D')
        break

w1:    0.999999915448461
EOF :D


In [10]:
sample_init.to_csv('sample_size_speedup'+ str(m) + file_to_read, index = False)